## Environment Setup

In [1]:
from copy import deepcopy

from my_entries.likehood_agent import LikelihoodAgent, CountFeatureProvider
from recogym.evaluate_agent import plot_verify_agents, verify_agents
%matplotlib inline
import gym
import matplotlib.pyplot as plt

from recogym import env_1_args, Configuration

# Set style for pretty plots
plt.style.use('ggplot')

products = 40

offline_users = 1000
online_users = 100

# You can overwrite environment arguments here:

env_1_args['random_seed'] = 42
env_1_args['num_products']= products
env_1_args['phi_var']=0.0
env_1_args['number_of_flips']=1

# env_1_args['sigma_mu_organic'] = 0.0
# env_1_args['sigma_omega']=0
# env_1_args['normalize_beta']=True

# Initialize the gym for the first time by calling .make() and .init_gym()

#---------------------------------------------
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [6, 3]

env_1_args['number_of_flips'] = 0
env_1_args['sigma_mu_organic'] = 0.0
env_1_args['sigma_omega'] = 1
env_1_args['random_seed'] = 42
env_1_args['num_products'] = products
env_1_args['K'] = 5
env_1_args['number_of_flips'] = 5
#---------------------------------------------

env = gym.make('reco-gym-v1')
env.init_gym(env_1_args)

env.reset()


/workspace/ML/reco-gym/recogym/envs/abstract.py:54: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


8. Define Likelihood Agent

In [2]:
config = Configuration(env_1_args)

count_product_views_feature_provider = CountFeatureProvider(config)

likelihood_agent = LikelihoodAgent(count_product_views_feature_provider)

lh_data = deepcopy(env).generate_logs(offline_users)

likelihood_agent.train(lh_data)  # ------- DIRECT TRAIN CALL

bandits = lh_data[lh_data['z'] == 'bandit']
successes = bandits[bandits['c'] == 1].shape[0]
failures = bandits[bandits['c'] == 0].shape[0]

Organic Users: 0it [00:00, ?it/s]
Users:   0%|          | 0/1000 [00:00<?, ?it/s]

# ------------------------------------------------------------------------------------
#  GENERATE LOGS FOR AGENT None
# ------------------------------------------------------------------------------------


Users: 100%|██████████| 1000/1000 [00:17<00:00, 56.17it/s]



LikelihoodAgent train() num_products 40

run model.fit rewards size 80329


KeyboardInterrupt: 

In [ ]:
#ALL LEN

len(lh_data)

In [ ]:
#BANDITS LEN
len(bandits)

In [ ]:
#SUCCESS
successes

In [ ]:
#FAILURES
failures


In [ ]:
result = verify_agents(
    env,
    number_of_users=online_users,
    agents={
        'Likelihood Agent': likelihood_agent,
    }
)

In [ ]:
result

In [ ]:
fig = plot_verify_agents(result)
plt.show()
